In [259]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 


# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

import sys
sys.path.append("../")
from src import soporte
pd.set_option('display.max_rows', None)

In [230]:
#Vamos a scrapear ecoalf

#Accedemos a la página
driver = webdriver.Chrome()
url = "https://ecoalf.com/collections/abrigos-y-chaquetas-210-hombre"
driver.get(url)
driver.maximize_window()
sleep(10)
driver.execute_script(f"window.scrollTo(0, {0,20});")

WebDriverWait(driver,5).until(EC.presence_of_element_located(("xpath","/html/body/div[1]/div/div[1]/div/div[2]/button[2]"))).click()
print("Hemos clickado en la x")

#Scrolleamos hasta el final para que cargue toda la página, de 200 en 200 para que cargue tranquilamente
altura_pagina = driver.execute_script("return document.body.scrollHeight")

while altura_pagina == altura_pagina_anterior:
    altura_pagina_anterior = altura_pagina
    for y in range(0, altura_pagina, 200):
        driver.execute_script(f"window.scrollTo(0, {y});")
        altura_pagina = driver.execute_script("return document.body.scrollHeight")
        sleep(2)



# Usamos BeautifulSoup para analizar el HTML de la página cargada
sopa_chaquetas_ecoalf = BeautifulSoup(driver.page_source, "html.parser")

# Extraemos todos los enlaces de chaquetas
nombre_chaqueta = sopa_chaquetas_ecoalf.find_all("a", class_="full-unstyled-link")

# Mostramos la cantidad de chaquetas encontradas
print(f"Cantidad de chaquetas encontradas: {len(nombre_chaqueta)}")

# Cerramos el navegador
driver.quit()


# #Sacamos las categorías y género
denominacion_url = url.rsplit("-")
genero = denominacion_url[-1]
categoria = denominacion_url[-3]

# Guardamos todas las cajas
caja = sopa_chaquetas_ecoalf.findAll("a", class_= "full-unstyled-link")

print(len(caja))


Hemos clickado en la x
Cantidad de chaquetas encontradas: 115
115


In [231]:

# Buscar todos los elementos <a> con la clase "full-unstyled-link"
enlaces = sopa_chaquetas_ecoalf.find_all("a", class_="full-unstyled-link")

# Extraer el atributo 'href' de cada enlace
links = [enlace['href'] for enlace in enlaces if 'href' in enlace.attrs]
links = links[14:]

links_completos = ["https://ecoalf.com" + i for i in links]

In [232]:
links_completos

['https://ecoalf.com/products/venuealf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/graysalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/livornoalf-jacket-man-black-aw24',
 'https://ecoalf.com/products/velanalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/pasualf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/venuealf-jacket-man-woodland-green-aw24',
 'https://ecoalf.com/products/vinsonalf-jacket-man-dark-olive-aw24',
 'https://ecoalf.com/products/new-icebergalf-jacket-man-woodland-green-aw24',
 'https://ecoalf.com/products/parkoalf-jacket-man-deepnavy-aw24',
 'https://ecoalf.com/products/vinsonalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/cartesalf-jacket-man-dark-olive-aw24',
 'https://ecoalf.com/products/new-vintagealf-jacket-man-anthracite-aw24',
 'https://ecoalf.com/products/graysalf-jacket-man-armygreen-aw24',
 'https://ecoalf.com/products/new-icebergalf-jacket-man-deep-navy-aw24',
 'https://ecoalf.com/products/pa

In [233]:
nombre_elemento =  []
precio_elemento =  []
detalles_elemento =  []

for single_link in links_completos:
    res_link = requests.get(single_link)
    sopa_elemento = BeautifulSoup(res_link.content, "html.parser")
    nombre_elemento.append(sopa_elemento.find_all("h1","product__title h4"))
    precio_elemento.append(sopa_elemento.find_all("div", class_="price__regular price-item price-item--regular"))
    detalles_elemento.append(sopa_elemento.find_all("div", class_="product__description rte"))

In [234]:
nombres_done = []
for lista in nombre_elemento:
    for i in lista:
        nombres_done.append(i.getText().strip())

nombres_done

['CHAQUETA VENUE AZUL MARINO',
 'CHAQUETA VENUE AZUL MARINO',
 'CHAQUETA GRAYS AZUL MARINO',
 'CHAQUETA GRAYS AZUL MARINO',
 'ABRIGO LIVORNO NEGRO',
 'ABRIGO LIVORNO NEGRO',
 'ABRIGO VELAN AZUL MARINO',
 'ABRIGO VELAN AZUL MARINO',
 'ABRIGO PASU AZUL MARINO',
 'ABRIGO PASU AZUL MARINO',
 'CHAQUETA VENUE VERDE OSCURO',
 'CHAQUETA VENUE VERDE OSCURO',
 'CHAQUETA VINSON VERDE',
 'CHAQUETA VINSON VERDE',
 'ABRIGO ICEBERG VERDE OSCURO',
 'ABRIGO ICEBERG VERDE OSCURO',
 'ABRIGO PARKO AZUL MARINO',
 'ABRIGO PARKO AZUL MARINO',
 'CHAQUETA VINSON AZUL MARINO',
 'CHAQUETA VINSON AZUL MARINO',
 'CHAQUETA CARTES VERDE',
 'CHAQUETA CARTES VERDE',
 'ABRIGO NEW VINTAGE GRIS OSCURO',
 'ABRIGO NEW VINTAGE GRIS OSCURO',
 'CHAQUETA GRAYS VERDE',
 'CHAQUETA GRAYS VERDE',
 'ABRIGO ICEBERG AZUL MARINO',
 'ABRIGO ICEBERG AZUL MARINO',
 'ABRIGO PARKO VERDE OSCURO',
 'ABRIGO PARKO VERDE OSCURO',
 'CHAQUETA ASP AZUL MARINO',
 'CHAQUETA ASP AZUL MARINO',
 'CHAQUETA ASP AZUL MARINO',
 'CHAQUETA ASP AZUL MARINO',


In [235]:
precios_done = []
for lista in precio_elemento:
    for i in lista:
        precios_done.append(i.getText().strip())

In [236]:
detalles_element = []
for lista in detalles_elemento:
    for i in lista:
        detalles_element.append(i.getText().strip())

nueva_lista = []
for i in detalles_element:
        
    nueva_lista.append(i.split("\n"))
  

descripcion = []
tejido = []
for i in nueva_lista:
    descripcion.append(i[0])
    tejido.append(i[2])



In [290]:
descripcion

['La chaqueta Venue está hecha de poliéster reciclado que procede de botellas de plástico a las que damos una segunda vida. Lleva capucha incorporada ajustable y entre sus propiedades técnicas destaca que es impermeable y transpirable; la chaqueta perfecta para los días de lluvia.',
 'La chaqueta Grays es monomaterial y está hecha de 100% poliéster reciclado procedente de botellas de plástico. Además, es transpirable y repelente al agua. Una chaqueta muy cómoda y perfecta para este invierno con la que combinarás estilo y sostenibilidad.',
 'El abrigo Livorno está hecho con nuestro Hilo del Mar procedente de botellas de plástico recicladas del fondo del mar a través del proyecto Upcycling the Oceans de la Fundación Ecoalf. Hemos recogido más de 1.450 toneladas del fondo del océano desde 2015 gracias al apoyo de más de 4.500 pescadores. Ayúdanos a seguir limpiando nuestros océanos.',
 'El abrigo Velan integra la tecnología innovadora dope dye que permite un proceso de teñido sin agua. En